In [1]:
import librosa

In [2]:
# import some libraries you maybe use
import torchvision # an useful library to help I/O (highly recommend). To install this, just do "pip install torchvision"
import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
print(np.__version__)
print(torch.__version__)
print(torchvision.__version__)
import os
import visdom
import scipy
import scipy.signal

1.17.4
1.0.1.post2
0.2.2


In [4]:
batch_size = 120

In [180]:
ROOT_DIR = './results'
TRAINING_NAME = 'ensemble_mp_mel'
OUTPUT_DIR = os.path.join(ROOT_DIR, TRAINING_NAME)
MODEL_PATH = os.path.join(OUTPUT_DIR, 'weight.pth')
RESULT_PATH = os.path.join(OUTPUT_DIR, 'result.csv')
print(RESULT_PATH)

./results/ensemble_mp_mel/result.csv


In [182]:
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)

In [81]:
ROOT_DIR = './results'
_TRAINING_NAME = 'resnet50_adam_phase'
_OUTPUT_DIR = os.path.join(ROOT_DIR, _TRAINING_NAME)
_MODEL_PATH = os.path.join(_OUTPUT_DIR, 'weight.pth')
_RESULT_PATH = os.path.join(_OUTPUT_DIR, 'result.csv')
_NP_PATH =  os.path.join(_OUTPUT_DIR, 'raw_result.npy')
print(_NP_PATH)

./results/resnet50_adam_phase/raw_result.npy


In [53]:
eTRAINING_NAME = 'resnet50_adam_10e3'
eOUTPUT_DIR = os.path.join(ROOT_DIR, eTRAINING_NAME)
eMODEL_PATH = os.path.join(eOUTPUT_DIR, 'weight.pth')
eRESULT_PATH = os.path.join(eOUTPUT_DIR, 'result.csv')
eNP_PATH =  os.path.join(eOUTPUT_DIR, 'raw_result.npy')
print(eNP_PATH)

./results/resnet50_adam_10e3/raw_result.npy


# Data loading and preprocessing
In order to train the model with training data, the first step is to read the data from your folder, database, etc. The below is just an example.

In [32]:
from torchvision.datasets import ImageFolder, DatasetFolder
from torchvision.transforms import Compose, ToTensor, Grayscale, Resize, Normalize
from torch.utils.data import DataLoader
import os
# Define path to your dataset
dataset = "./data" # the root folder
trainpath = os.path.join(dataset,"train") # train set
valpath = os.path.join(dataset,"val") # validation set

cut = lambda x: x[:11025]
norm =  lambda x: x.astype(np.float32) / np.max(x)
spct = lambda x: scipy.signal.spectrogram(x ,fs= 10e3,mode='magnitude')[2]
tri = lambda x: [x, x, x]
totensor = lambda x: torch.Tensor(x)

tsfm = Compose([
        cut, # rescale to -1 to 1
        norm, # rescale to -1 to 1
        spct, # MFCC 
        tri,
        totensor
        ])


nploader = np.load


In [33]:
traindata = DatasetFolder(root=trainpath, loader=nploader, transform=tsfm, extensions=['npy'])
valdata = DatasetFolder(root=valpath, loader=nploader, transform=tsfm, extensions=['npy'])

# Create a loader
trainloader = DataLoader(traindata,batch_size=batch_size,shuffle=True, pin_memory=True, num_workers=6)
valloader = DataLoader(valdata,batch_size=batch_size,shuffle=True,  pin_memory=True, num_workers=6)

In [34]:
print(traindata.classes) # show all classes
print(traindata.class_to_idx) # show the mapping from class to index.

['Frog1', 'Frog2', 'Frog3', 'Grylloidea1', 'Grylloidea2', 'Tettigonioidea1', 'Tettigonioidea2', 'drums_FloorTom', 'drums_HiHat', 'drums_Kick', 'drums_MidTom', 'drums_Ride', 'drums_Rim', 'drums_SmallTom', 'drums_Snare', 'guitar_3rd_fret', 'guitar_7th_fret', 'guitar_9th_fret', 'guitar_chord1', 'guitar_chord2']
{'Frog1': 0, 'Frog2': 1, 'Frog3': 2, 'Grylloidea1': 3, 'Grylloidea2': 4, 'Tettigonioidea1': 5, 'Tettigonioidea2': 6, 'drums_FloorTom': 7, 'drums_HiHat': 8, 'drums_Kick': 9, 'drums_MidTom': 10, 'drums_Ride': 11, 'drums_Rim': 12, 'drums_SmallTom': 13, 'drums_Snare': 14, 'guitar_3rd_fret': 15, 'guitar_7th_fret': 16, 'guitar_9th_fret': 17, 'guitar_chord1': 18, 'guitar_chord2': 19}


In [35]:
idx_to_class = {val: key for key, val in traindata.class_to_idx.items()} # build an inverse mapping for later use
print(idx_to_class)

{0: 'Frog1', 1: 'Frog2', 2: 'Frog3', 3: 'Grylloidea1', 4: 'Grylloidea2', 5: 'Tettigonioidea1', 6: 'Tettigonioidea2', 7: 'drums_FloorTom', 8: 'drums_HiHat', 9: 'drums_Kick', 10: 'drums_MidTom', 11: 'drums_Ride', 12: 'drums_Rim', 13: 'drums_SmallTom', 14: 'drums_Snare', 15: 'guitar_3rd_fret', 16: 'guitar_7th_fret', 17: 'guitar_9th_fret', 18: 'guitar_chord1', 19: 'guitar_chord2'}


In [36]:
correct_idx2class = {9: 'Frog1', 10: 'Frog2', 19: 'Frog3', 3: 'Grylloidea1', 14: 'Grylloidea2', 0: 'Tettigonioidea1', 1: 'Tettigonioidea2', 11: 'drums_FloorTom', 5: 'drums_HiHat', 6: 'drums_Kick', 4: 'drums_MidTom', 16: 'drums_Ride', 13: 'drums_Rim', 7: 'drums_SmallTom', 2: 'drums_Snare', 15: 'guitar_3rd_fret', 12: 'guitar_7th_fret', 18: 'guitar_9th_fret', 17: 'guitar_chord1', 8: 'guitar_chord2'}
#print(correct_idx2class)

In [37]:
correct_class2idx = {val: key for key, val in correct_idx2class.items()}
#print(correct_class2idx)

In [38]:
corrected_idx2idx = {val: correct_class2idx[key] for key, val in traindata.class_to_idx.items()}
print(corrected_idx2idx)

{0: 9, 1: 10, 2: 19, 3: 3, 4: 14, 5: 0, 6: 1, 7: 11, 8: 5, 9: 6, 10: 4, 11: 16, 12: 13, 13: 7, 14: 2, 15: 15, 16: 12, 17: 18, 18: 17, 19: 8}


# Build an example network
If you're unfamiliar with this part, please see the HW1 tutorial.

In [39]:
import utils.resnet as resnet
#model = resnet.resnet50(num_classes= len(traindata.classes))

# Load model

In [78]:
def load_model(model,filename):
    model.load_state_dict(torch.load(filename))
    return model
net = resnet.resnet50(num_classes= len(traindata.classes))
enet = resnet.resnet50(num_classes= len(traindata.classes))
net = load_model(net, _MODEL_PATH)
enet = load_model(enet, eMODEL_PATH)
# Whether to use GPU or not?
device = 'cpu'
if torch.cuda.is_available():
    device = 'cuda'
else: 
    device = 'cpu'
print("use",device,"now!")
net.to(device)
enet.to(device)

AttributeError: 'ResNet' object has no attribute 'softmax'

# Evaluate on validation data

In [41]:
net.eval()
enet.eval()
correct = 0
with torch.no_grad():
    for batch_idx, (data, target) in enumerate(valloader):
        #print(type(data))
        data = data.to(device)
        target = target.to(device)
        #output = net(data)
        eoutput = enet(data)
        #en_output = (output + eoutput)/2.0
        pred = eoutput.data.max(1, keepdim=True)[1]
        correct += pred.eq(target.data.view_as(pred)).sum()
    acc = correct.item() / len(valloader.dataset)
print("Validation Classification Accuracy: %f"%(acc))

Validation Classification Accuracy: 0.976764


In [74]:
eoutput

tensor([[-34.9170, -32.2825,  -1.1669,  -1.8453, -22.7595,  -5.7541, -14.6282,
           7.3866, -54.9194, -34.4602,   5.2446, -60.6533, -79.9013,   9.8023,
         -24.1531,  20.9104, -24.4147,  12.3602,  -2.0933,  18.4360]],
       device='cuda:0')

In [65]:
test_data = np.load('./data/test.npy', allow_pickle=True)
print(len(test_data))

2387


In [66]:
t_spec = []
for t in test_data:
    test = cut(t)
    test = norm(test)
    test = spct(test) 
    test = tri(test)
    test = totensor(test)
    t_spec.append(test)

In [67]:
tensor_x = torch.stack(t_spec)

In [68]:
test_dataset = torch.utils.data.TensorDataset(tensor_x) # create your datset
test_dataloader = torch.utils.data.DataLoader(test_dataset) # create your dataloader

In [69]:
enet.eval()
result = []
raw_result = []
with torch.no_grad():
    for batch_idx, (data, ) in enumerate(test_dataloader):  
        data = data.to(device)
        #target = target.to(device)
        #output = enet(data)
        eoutput = enet(data)
        #en_output = (output + eoutput)/2
        pred = eoutput.data.max(1, keepdim=True)[1]
        #correct += pred.eq(target.data.view_as(pred)).sum()
        result = result + list(pred.cpu().numpy().ravel())
        raw_result.append(list(eoutput.cpu().numpy().ravel()))
    #acc = correct.item() / len(valloader.dataset)
#print("Validation Classification Accuracy: %f"%(acc))

In [70]:
np.save(eNP_PATH, np.asarray(raw_result))

In [40]:
result = [corrected_idx2idx[idx] for idx in result]

In [41]:
df_dict = {'id': list(range(0 ,len(test_data))), 'category':result}

In [42]:
import pandas as pd
df = pd.DataFrame(df_dict)

In [43]:
df.head()

,id,category
0,0,1
1,1,19
2,2,15
3,3,17
4,4,8


In [44]:
print(RESULT_PATH)

./results/ensemble_mp/result.csv


In [45]:
df.to_csv(RESULT_PATH, index=False)

# NUMPY

In [209]:
ROOT_DIR = './results'
TRAINING_NAME = 'ensemble_mp_melnpad_phasenormpad'
OUTPUT_DIR = os.path.join(ROOT_DIR, TRAINING_NAME)
MODEL_PATH = os.path.join(OUTPUT_DIR, 'weight.pth')
RESULT_PATH = os.path.join(OUTPUT_DIR, 'result.csv')
print(RESULT_PATH)

./results/ensemble_mp_melnpad_phasenormpad/result.csv


In [210]:
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)

In [211]:
names = {'resnet50_phase_decay_norm_pad':0.5, 'resnet50_meldata':0.5}
ROOT_DIR = './results'
OUTPUT_DIR = os.path.join(ROOT_DIR, TRAINING_NAME)
_NP_PATH =  os.path.join(_OUTPUT_DIR, 'raw_result.npy')

In [212]:
probs = None
for name in names:
    OUTPUT_DIR = os.path.join(ROOT_DIR, name)
    rnp = np.load(os.path.join(OUTPUT_DIR, 'raw_result.npy'))
    if probs is None:
        probs=nn.functional.softmax(torch.tensor(rnp), dim=1)*names[name]
    else:
        probs+=nn.functional.softmax(torch.tensor(rnp), dim=1)*names[name]

In [213]:
#eresult = np.load(eNP_PATH)

In [214]:
#_result = np.load(_NP_PATH)

In [215]:
#_s = nn.functional.softmax(torch.tensor(_result), dim=1)
#es = nn.functional.softmax(torch.tensor(eresult), dim=1)

In [216]:
#nn.functional.softmax(torch.tensor(_result[100]), dim=0)

In [217]:
#_sum = (_s + es)/n_of_result

In [218]:
result = probs.data.max(1, keepdim=True)[1].cpu().numpy().ravel()

In [219]:
result = [corrected_idx2idx[idx] for idx in result]

In [220]:
df_dict = {'id': list(range(0 ,len(test_data))), 'category':result}

In [221]:
import pandas as pd
df = pd.DataFrame(df_dict)

In [222]:
df.head()

,id,category
0,0,18
1,1,2
2,2,18
3,3,8
4,4,8


In [223]:
df.to_csv(RESULT_PATH, index=False)

In [224]:
torch.cuda.empty_cache()